In [1]:
from MFEA_lib.model import Model_Offical,Model_Test
from MFEA_lib.model.utils import *
from MFEA_lib.operators.Crossover import *
from MFEA_lib.operators.Mutation import *
from MFEA_lib.operators.Selection import *
from MFEA_lib.operators.Search import *
from MFEA_lib.tasks.Benchmark.Funcs import WCCI22_benchmark

In [2]:
class IndividualDE(Individual):
    def __init__(self, genes, dim= None) -> None:
        super().__init__(genes, dim)
        if genes is None:
            self.genes: np.ndarray = np.random.rand(dim)
        self.transfer =False
        self.d_rank = 1

In [3]:
ls_benchmark = []
ls_IndClass = []
for i in range(1, 11, 1):
    t, ic = WCCI22_benchmark.get_50tasks_benchmark(i)
    ls_benchmark.append(t)
    ls_IndClass.append(IndividualDE)

name_benchmark = np.arange(len(ls_benchmark)) + 1

In [4]:
model = MultiBenchmark(
    ls_benchmark= ls_benchmark,
    name_benchmark= name_benchmark,
    ls_IndClass= ls_IndClass,
    model = Model_Test
)
model.compile(
    crossover= SBX_Crossover(nc = 2),
    mutation = Polynomial_Mutation(nm = 5),
    selection= ElitismSelection(random_percent= 0.),
)
model.fit(
    nb_inds_each_task=100,
    nb_inds_min=30,
    nb_generations=1000,
    LSA = True,
    evaluate_initial_skillFactor= True
)
a = model.run(
    nb_run= 1,
    save_path= "./model_saved/__static__/WCCI22_manytask/model_test/model_benchmark_id_"
)

Time: 11m 10.39s  100 % [====================>]  Pop_size: 4.65E+03  ,  Cost: 4.75E-07  4.53E-07  4.77E-07  4.95E-07  4.82E-07  4.85E-07  4.83E-07  4.80E-07  4.90E-07  4.94E-07  4.75E-07  4.74E-07  4.79E-07  4.96E-07  4.94E-07  4.65E-07  4.83E-07  4.96E-07  4.22E-07  4.83E-07  4.93E-07  4.77E-07  4.98E-07  4.66E-07  4.80E-07  4.97E-07  4.75E-07  4.70E-07  4.59E-07  4.71E-07  4.79E-07  4.97E-07  4.93E-07  4.62E-07  4.65E-07  4.86E-07  4.85E-07  4.64E-07  4.94E-07  4.92E-07  4.97E-07  4.96E-07  4.88E-07  4.98E-07  4.91E-07  4.92E-07  4.84E-07  4.86E-07  4.87E-07  4.52E-07  ,  DONE!
Saved
Time: 12m 33.15s  100 % [====================>]  Pop_size: 4.65E+03  ,  Cost: 4.92E+01  1.64E+02  7.76E+01  4.47E+01  1.68E+02  4.69E+01  4.92E+01  4.84E+01  4.64E+01  4.62E+01  4.71E+01  9.56E+01  4.92E+01  4.67E+01  4.34E+01  4.07E+01  4.78E+01  4.75E+01  4.72E+01  4.52E+01  4.80E+01  4.89E+01  4.67E+01  4.83E+01  4.59E+01  4.72E+01  4.90E+01  7.73E+01  1.02E+02  4.87E+01  4.61E+01  4.90E+01  3.49E+01 

Write the result complex benchmark to text file
---------------

In [ ]:
# def write_res(res, path_out):
#     with open(path_out, 'w') as fi:
#         for i in res:
#             fi.write(str(int(i[0])) + ", ")
#             for j in range(1, len(i)-1):
#                 fi.write(str(i[j])+", ")
#             fi.write(str(i[len(i)-1])+"\n")

# len_out_complex = 100

# for id in range(1, 11):

#     model_mbm = loadModel(f"model_saved\__static__\WCCI22_complex\model_official\model_benchmark_id_{id}.mso", ls_tasks=WCCI22_benchmark.get_complex_benchmark(id))

#     res = model_mbm.ls_model[0].history_cost[:len_out_complex]
#     for i in range(1,model_mbm.nb_run):
#         res = np.concatenate((res, np.array(model_mbm.ls_model[i].history_cost[:100])), 1)
#     res.shape[0]
#     num_evals = (np.arange(1, res.shape[0] + 1) * 1000 * len(model_mbm.tasks)).reshape(-1,1)
#     out = np.concatenate((num_evals, res), 1)



#     write_res(out, f"./result/MTO-complex/MTOSOO_P{id}.txt")